In [ ]:
# 导入Py_Service
import asyncio
import websockets
import nest_asyncio
import sys
import os

sys.path.append("/Users/nobody/Documents/develop/GitHub/Services/PY_Service")
from utils import pyServiceUtils
from utils import cmdUtils

from Code.app.services.LogUtils.JsLogUtils import JsLogUtils

_jsLogUtils : JsLogUtils = pyServiceUtils.getSubSvrByName("Code", "LogUtils","JsLogUtils")

from BB.app.services.BBTs.BBTs_V8_Debugger import BBTs_V8_Debugger

_v8_Debugger: BBTs_V8_Debugger = pyServiceUtils.getSubSvrByName("BB", "BBTs", "BBTs_V8_Debugger")

nest_asyncio.apply()


async def listen_for_connection(uri, timeout):
    start_time = asyncio.get_event_loop().time()
    while True:
        try:
            async with websockets.connect(uri) as websocket:
                print("发现服务")
                return
        except (ConnectionRefusedError, OSError):
            if (asyncio.get_event_loop().time() - start_time) > timeout:
                print("未能在指定时间内发现指定端口的 websocket 服务")
                sys.exit(1)
            else:
                await asyncio.sleep(1)


async def is_websocket_service_available(uri):
    try:
        async with websockets.connect(uri) as websocket:
            return True
    except (ConnectionRefusedError, OSError):
        return False


async def waitForSecond(sec: int):
    await asyncio.sleep(sec)


from utils.CompanyUtil import Company_BB_Utils
import os

# 确保 ts 监控开启，这样生成完毕，直接就拷贝到Unity中了
_exwCmd = "chmod +x tsc"
_watchCmd = "tsc --watch --project tsconfig.json"
_specificText = "Found 0 errors. Watching for file changes."
_tsProject = os.path.join(Company_BB_Utils.getSLGProjectPath(), "project_ts")
_uri = "ws://localhost:8080"

In [ ]:

# ts 生成 js 代码，覆盖掉之前添加过日志的
cmdUtils.run_command_threading(
    _watchCmd,
    _tsProject,
    15,
    _specificText,
)

# 关闭掉监控 ts 变化的进程
if cmdUtils.isCmdRunning(_watchCmd):
    print(f"kill : {_watchCmd}")
    cmdUtils.killCmd(_watchCmd)

In [ ]:
# 为 导出的 js 文件添加日志
_distPath = os.path.join(_tsProject, "dist")
_jsFolderRelativePath = "Game/Module/"
# _jsFolderRelativePath = "Game/Module/Guide/"
_logUtilRelativePath = "Framework/ABDebug/LogUtils.js"
_jsLogUtils.analyseJsFolder(_distPath, _jsFolderRelativePath, _logUtilRelativePath)

In [ ]:

# 再次重启 Unity 上的 js 环境
_v8_Debugger.clearLogs()  # 清理日志

_V8_is_online = await is_websocket_service_available(_uri)
if _V8_is_online:
    _v8_Debugger.reset()  # 重置环境
    await waitForSecond(5)  # 等主界面启动

await listen_for_connection(_uri, 20)  # 等待 Unity 中 V8 启动
